In [19]:
import pandas as pd
import plotly.express as px
import country_converter as coco

In [21]:
df = pd.read_csv('./data/salaries.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,MI,FT,Data Engineer,100000,AUD,75050,AU,50,AU,L
1,2022,SE,FT,Data Engineer,225000,USD,225000,US,0,US,M
2,2022,SE,FT,Data Engineer,184100,USD,184100,US,0,US,M
3,2022,SE,FT,Data Scientist,185900,USD,185900,US,0,US,M
4,2022,SE,FT,Data Scientist,129300,USD,129300,US,0,US,M


In [22]:
df_source_data.shape

(1186, 11)

In [36]:
df.isna().sum()

work_year             0
job_title             0
salary_in_usd         0
employee_residence    0
company_location      0
dtype: int64

In [23]:
# Convert countries to ISO 3 code for possible merge later
df['employee_residence'] = coco.convert(names=df['employee_residence'], to="ISO3")
df.drop(['experience_level', 'employment_type', 'remote_ratio','company_size', 'salary', 'salary_currency'], inplace=True, axis=1)

df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,MI,FT,Data Engineer,100000,AUD,75050,AUS,50,AU,L
1,2022,SE,FT,Data Engineer,225000,USD,225000,USA,0,US,M
2,2022,SE,FT,Data Engineer,184100,USD,184100,USA,0,US,M
3,2022,SE,FT,Data Scientist,185900,USD,185900,USA,0,US,M
4,2022,SE,FT,Data Scientist,129300,USD,129300,USA,0,US,M


In [25]:
# Read BigMac index Data
df_big_mac = pd.read_csv('https://raw.githubusercontent.com/TheEconomist/big-mac-data/master/output-data/big-mac-full-index.csv')

# Convert date to datetime for easier filtering
df_big_mac.date = pd.to_datetime(df_big_mac.date)

# Filter BigMac data to only use dates after 2018
df_big_mac = df_big_mac[df_big_mac['date'] > '2019']
df_big_mac.reset_index(drop=True, inplace=True)
#Group bigmac by year

df_big_mac = pd.DataFrame(df_big_mac.groupby([df_big_mac.date.dt.year, df_big_mac.iso_a3]).mean())
df_big_mac.reset_index( inplace=True)
df_big_mac.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_bigmac,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
0,2000-04-01,ARG,ARS,Argentina,2.50,1.00,2.500000,0.11607,0.05007,-0.16722,-0.09864,1.09091,7803.328512,1.922652,0.39117,NaN,-0.06626,0.10096,0.97153
1,2000-04-01,AUS,AUD,Australia,2.59,1.68,1.541667,-0.31176,-0.35246,-0.48645,-0.44416,0.28939,29144.876973,2.301550,-0.28335,NaN,-0.51898,-0.43285,0.01563
2,2000-04-01,BRA,BRL,Brazil,2.95,1.79,1.648045,-0.26427,-0.30778,-0.45102,-0.40581,0.37836,4822.738983,1.869734,-0.05696,NaN,-0.36704,-0.25369,0.33645
3,2000-04-01,CAN,CAD,Canada,2.85,1.47,1.938776,-0.13448,-0.18566,-0.35417,-0.30099,0.62152,26087.329235,2.247266,-0.07698,NaN,-0.38047,-0.26953,0.30809
4,2000-04-01,CHE,CHF,Switzerland,5.90,1.70,3.470588,0.54937,0.45774,0.15609,0.25130,1.90267,23872.716095,2.207948,0.68172,NaN,0.12876,0.33090,1.38330


In [29]:
# Filter bigmac for relevant data
merge_df = df_big_mac.filter(['date', 'iso_a3','adj_price'])
merge_df.dropna(inplace=True)

#merge filtered salary data with bigmac data
new_df = pd.merge(df,merge_df, left_on=['work_year','employee_residence'], right_on=['date','iso_a3']).drop(columns=['date','iso_a3'])

#calculate number of bigmacs a salary can by in a country
new_df['num_bigmacs'] = new_df['salary_in_usd'] / new_df['adj_price']
new_df.head()

,date,iso_a3,adj_price
1,2019,ARG,3.240098
2,2019,AUS,4.385329
5,2019,BRA,3.138321
6,2019,CAN,3.985136
7,2019,CHE,4.484086


In [91]:
#create function to draw a map 
def create_map(df, measurement, location):
    '''
    Takes a Dataframe and two column names to aggregate by measurement and map aggregates by location
    The Median measurement is shown
    '''
    bigmac_location = df.groupby([measurement, location]).size().reset_index()
    average = bigmac_location.groupby(location).median().reset_index()

    fig = px.choropleth(locations=average[location],
                        color=average[measurement],
                        color_continuous_scale=px.colors.sequential.haline,
                        template='plotly_dark',
                        title = f'{measurement} for {location}')
    fig.update_layout(font = dict(size=15,family="Helvetica"))
    fig.show()

In [87]:
create_map(new_df, 'num_bigmacs','employee_residence')

In [48]:
# What's going on in Russia?

new_df[new_df['employee_residence'] == 'RUS']

,work_year,job_title,salary_in_usd,employee_residence,company_location,adj_price,num_bigmacs
947,2022,Data Scientist,48000,RUS,US,3.913002,12266.795985
985,2021,Head of Data Science,85000,RUS,RU,3.815083,22279.988843
986,2021,Head of Data,230000,RUS,RU,3.815083,60287.028635
1026,2020,Computer Vision Engineer,60000,RUS,US,3.622955,16561.066649


In [51]:
#read data table from url

url = 'https://www.worlddata.info/cost-of-living.php'
#Convert table to data frame
cost_df = pd.read_html(url)[0]
#Convert country names to ISO code
cost_df['iso_a3'] =  coco.convert(names=cost_df['Country'], to="ISO3")
#Merge with salary data
df_cost = pd.merge(df,cost_df, left_on='employee_residence', right_on='iso_a3')
#drop irrelevant columns
df_cost.drop(['Ø Monthly income','iso_a3'], axis=1, inplace=True)
#calculate adjusted salaries
df_cost['adj_salary_purch'] = (df_cost['salary_in_usd'] / df_cost['Purchasing power index']*100)
df_cost['adj_salary_cost'] = (df_cost['salary_in_usd'] / df_cost['Cost index']*100)

df_cost.head()


,Rank,Country,Cost index,Ø Monthly income,Purchasing power index
0,1,Bermuda *,157.6,"9,712 USD",105.0
1,2,Switzerland,142.4,"7,530 USD",90.1
2,3,Cayman Islands *,137.9,"5,281 USD",65.2
3,4,Israel,130.2,"4,130 USD",54.1
4,5,Iceland,128.0,"5,368 USD",71.5


In [89]:
create_map(df_cost, 'adj_salary_purch','employee_residence')

In [90]:
create_map(df_cost, 'adj_salary_cost','employee_residence')